In [ ]:
import math
import pandas as pd
import plotly.graph_objects as go
import numpy as np

Dudas:

- cómo se comparte una casa con una hipoteca?

- las dos personas serian responsables de la hipoteca y de la escritura de la vivienda.
- las dos personas que
- 30 años. min 5. amortizaciones parciales gratuitas. amortizaciones totales 2%.

3.11% TIN,  TAE: 3.49%: revisable.

314 itp + gastos de notacio 38900k. (financiacion de gastos).

2.61% TIN,  TAE: 3.40% bonificado: domicialidao nomina, seguro de vida, y seguro hogar.

fija 561,21 (2.61%, 3.18%)
598 (3.11, 3.39%)


amortizaciones parciales.


openbank.es: hipotecas.

la amortizacion inmediata


- qué comisiones se aplican para cada operación (amortización, cancelación, traspaso, refinanciación...)?

- qué gastos se incluyen dentro del tae?

- cuánto itp te dan si solo pago la mitad de la casa?

# Máximo precio de piso según sueldo

In [ ]:
def maximo_precio_piso_segun_sueldo(sueldo_neto_mensual,
                                    relacion_cuota_sueldo,
                                    porcentaje_entrada,
                                    tasa_interes,
                                    plazo):
    """
    :param sueldo_neto_mensual: sueldo neto mensual
    :param relacion_cuota_sueldo: maxima fraccion del sueldo neto mensual que puede suponer la cuota mensual
    :param porcentaje_entrada: porcentaje del valor total del inmueble que debe aportar el comprador
    :param tasa_interes: tasa de interes fijo del prestamo bancario
    :param plazo: tiempo de duracion del prestamo hipotecario (en meses)
    :return: Maximo importe total de la vivienda a adquirir a partir del sueldo
    """

    exponencial = (1. + tasa_interes / (12. * 100.))**(-plazo)
    factor = tasa_interes / (12. * 100.) * 1. / (1. - exponencial)
    cuota_mensual = sueldo_neto_mensual * relacion_cuota_sueldo #  = capital_pendiente * factor

    capital_pendiente = cuota_mensual / factor

    precio_piso = capital_pendiente / (1. - porcentaje_entrada / 100.)

    return precio_piso

In [ ]:
# Parametros generales de la hipoteca: tiempos en annus, tasa de interes fijo,
# porcentaje necesario de entrada de hipoteca, porcentaje de la cuota mensual
# con respeto al sueldo y precio total del piso derivado del sueldo neto mensual

t_hipoteca = 30.
tasa_interes = 2.0
porcentaje_entrada = 20.
relacion_cuota_sueldo = 1./3.

# Ejemplo: sueldo neto mensual de 2000 euros
sueldo_neto_mensual = 4000.

precio_piso = maximo_precio_piso_segun_sueldo(sueldo_neto_mensual=sueldo_neto_mensual,
                                              relacion_cuota_sueldo=relacion_cuota_sueldo,
                                              porcentaje_entrada=porcentaje_entrada,
                                              tasa_interes=tasa_interes,
                                              plazo=t_hipoteca * 12)

In [ ]:
print('El precio total de la vivienda no puede superar los: {} euros'.format(int(precio_piso)))

El precio total de la vivienda no puede superar los: 450914 euros


# Cálculo de Hipotecas:

### Funciones auxialiares

In [ ]:
def calcular_plazo(capital, tasa, cuota):
    """Calcula los meses restantes para pagar el préstamo con validación"""
    if capital <= 0:
        return 0

    tasa_mensual = tasa / 1200
    if cuota <= capital * tasa_mensual:
        raise ValueError("La cuota no cubre los intereses mínimos")

    plazo = -math.log(1 - capital * tasa_mensual / cuota) / math.log(1 + tasa_mensual)
    return max(math.ceil(plazo), 1)  # Nunca menos de 1 mes


def cuota_mensual(capital, tasa, plazo):
    """Calcula la cuota mensual con validación de parámetros"""
    if plazo <= 0:
        raise ValueError("El plazo debe ser mayor a 0 meses")
    if capital <= 0:
        return 0  # Si no hay capital, no hay cuota

    tasa_mensual = tasa / 1200
    return (capital * tasa_mensual) / (1 - (1 + tasa_mensual)**-plazo)


def intereses_mensuales(capital_pendiente, tasa):
    """Calcula los intereses mensuales"""
    tasa_mensual = tasa / 1200
    return capital_pendiente * tasa_mensual

## A tipo fijo

In [ ]:
def simulacion_hipoteca_simple(capital_inicial, tasa, plazo_inicial, cuota_inicial):
    """
    Simulación simplificada de hipoteca sin amortizaciones anticipadas.

    Args:
        capital_inicial (float): Capital inicial del préstamo.
        tasa (float): Tasa de interés anual en porcentaje (ej: 3.5 para 3.5%).
        plazo_inicial (int): Plazo inicial del préstamo en meses.
        cuota_inicial (float): Cuota mensual inicial del préstamo.

    Returns:
        pd.DataFrame: DataFrame con el detalle de la simulación mes a mes.
    """

    # Validaciones iniciales generales
    if capital_inicial <= 0:
        raise ValueError("El capital inicial debe ser positivo")
    if plazo_inicial <= 0:
        raise ValueError("El plazo inicial debe ser positivo")
    if cuota_inicial <= 0:
        raise ValueError("La cuota inicial debe ser positiva")


    registros = []
    capital_pendiente = capital_inicial
    cuota_mensual_fija = cuota_inicial # Cuota fija para hipoteca simple
    mes_actual = 0

    for mes in range(1, plazo_inicial + 1):
        mes_actual += 1

        if capital_pendiente <= 0:
            registros.append({
                'Mes': mes_actual,
                'Capital_pendiente': 0,
                'Cuota_mensual': 0,
                'Intereses_mensuales': 0,
                'Amortizacion_mensual': 0
            })
            break  # Finalizar si ya se pagó todo


        interes = intereses_mensuales(capital_pendiente, tasa)
        amortizacion = min(cuota_mensual_fija - interes, capital_pendiente)  # No sobrepagar

        registros.append({
            'Mes': mes_actual,
            'Capital_pendiente': capital_pendiente,
            'Cuota_mensual': cuota_mensual_fija,
            'Intereses_mensuales': interes,
            'Amortizacion_mensual': amortizacion
        })

        capital_pendiente -= amortizacion


    return pd.DataFrame(registros)


def calcular_intereses_totales_simple(capital_inicial, tasa, plazo_inicial, cuota_inicial):
    """
    Calcula los intereses totales de una hipoteca simple sin amortizaciones anticipadas.

    Args:
        capital_inicial (float): Capital inicial del préstamo.
        tasa (float): Tasa de interés anual en porcentaje.
        plazo_inicial (int): Plazo inicial del préstamo en meses.
        cuota_inicial (float): Cuota mensual inicial del préstamo.

    Returns:
        float: Intereses totales pagados durante la vida del préstamo.
    """
    df_simulacion = simulacion_hipoteca_simple(capital_inicial, tasa, plazo_inicial, cuota_inicial)
    intereses_totales = df_simulacion['Intereses_mensuales'].sum()
    return intereses_totales


def plot_hipoteca_simple(df_hipoteca_simple):

    # --- Plotly Interactive Plot ---
    fig = go.Figure()


    # Hipoteca Simple
    fig.add_trace(go.Scatter(x=df_hipoteca_simple['Mes'],
                            y=df_hipoteca_simple['Cuota_mensual'],
                            mode='lines',
                            line=dict(color='blue'),
                            name='Cuota Mensual'))

    fig.add_trace(go.Scatter(x=df_hipoteca_simple['Mes'],
                            y=df_hipoteca_simple['Amortizacion_mensual'],
                            mode='lines',
                            line=dict(color='red'),
                            name='Amortización Mensual'))

    fig.add_trace(go.Scatter(x=df_hipoteca_simple['Mes'],
                            y=df_hipoteca_simple['Intereses_mensuales'],
                            mode='lines',
                            line=dict(color='black'),
                            name='Intereses Mensuales'))


    fig.update_layout(
        xaxis_title="Mes",
        yaxis_title="Importe (euros)",
        font_family="Courier New",
        font_color="black",
        title_font_family="Times New Roman",
        title_font_color="black",
        legend_title_font_color="black",
        title='Simulación de Hipoteca Simple' # Título más descriptivo
    )

    fig.update_layout(legend=dict(
        yanchor="bottom", #  La leyenda ahora se ancla en la parte inferior
        y=0.02,        # Ajustamos la posición vertical ligeramente desde la parte inferior
        xanchor="left",
        x=0.01,
        bgcolor="white",
        bordercolor="black",
        borderwidth=1,
        font_family="Arial",
    ))

    fig.show()



# Ejemplo de uso de la simulación de hipoteca simple:
capital_inicial_simple = 200000
tasa_simple = 3.22
plazo_inicial_simple = 20 * 12 # 30 años en meses
cuota_inicial_simple = cuota_mensual(capital_inicial_simple, tasa_simple, plazo_inicial_simple)

df_hipoteca_simple = simulacion_hipoteca_simple(capital_inicial_simple, tasa_simple, plazo_inicial_simple, cuota_inicial_simple)
intereses_totales_simple = calcular_intereses_totales_simple(capital_inicial_simple, tasa_simple, plazo_inicial_simple, cuota_inicial_simple)

print("\n--- Simulación de Hipoteca Simple ---")
print(f"Capital inicial: {capital_inicial_simple:.2f} euros")
print(f"Tasa de interés anual: {tasa_simple:.2f}%")
print(f"Plazo inicial: {plazo_inicial_simple // 12} años")
print(f"Cuota mensual inicial: {cuota_inicial_simple:.2f} euros")

print(f"Intereses totales a pagar (hipoteca simple): {intereses_totales_simple:.2f} euros")
print("\nDataFrame de la simulación de hipoteca simple:")
#print(df_hipoteca_simple)

plot_hipoteca_simple(df_hipoteca_simple)


--- Simulación de Hipoteca Simple ---
Capital inicial: 200000.00 euros
Tasa de interés anual: 3.22%
Plazo inicial: 20 años
Cuota mensual inicial: 1131.35 euros
Intereses totales a pagar (hipoteca simple): 71524.10 euros

DataFrame de la simulación de hipoteca simple:


In [ ]:
def simulacion_intereses_vs_plazo(capital_inicial, tasa, plazos_en_años):
    """
    Simula y grafica la relación entre el plazo de la hipoteca y los intereses totales pagados.

    Args:
        capital_inicial (float): Capital inicial fijo de la hipoteca.
        tasa (float): Tasa de interés anual fija (en porcentaje).
        plazos_en_años (list): Lista de plazos de hipoteca en años a simular.

    Returns:
        plotly.graph_objects.Figure: Figura de Plotly con el gráfico de línea.
    """
    resultados_intereses = []
    plazos_en_meses_lista = []

    for plazo_años in plazos_en_años:
        plazo_meses = plazo_años * 12
        plazos_en_meses_lista.append(plazo_meses)
        cuota = cuota_mensual(capital_inicial, tasa, plazo_meses)
        intereses_totales = calcular_intereses_totales_simple(capital_inicial, tasa, plazo_meses, cuota)
        resultados_intereses.append(intereses_totales)

    df_resultados_plazo = pd.DataFrame({
        'Plazo_en_Años': plazos_en_años,
        'Plazo_en_Meses': plazos_en_meses_lista,
        'Intereses_Totales': resultados_intereses
    })

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_resultados_plazo['Plazo_en_Años'],
                             y=df_resultados_plazo['Intereses_Totales'],
                             mode='lines+markers',
                             line=dict(color='blue'),
                             marker=dict(size=8, symbol='circle'),
                             name='Intereses Totales'))

    fig.update_layout(
        title='Relación entre Plazo de Hipoteca e Intereses Totales Pagados',
        xaxis_title='Plazo de la Hipoteca (Años)',
        yaxis_title='Intereses Totales Pagados (euros)',
        font_family="Courier New",
        font_color="black",
        title_font_family="Times New Roman",
        title_font_color="black",
        legend_title_font_color="black",
    )

    fig.show()
    return fig, df_resultados_plazo


# --- Simulación de Intereses Totales vs Plazo ---
plazos_a_simular_años = [1, 10, 15, 20, 25, 30, 35, 40] # Plazos en años a simular
fig_intereses_plazo, df_intereses_plazo = simulacion_intereses_vs_plazo(capital_inicial_simple, tasa_simple, plazos_a_simular_años)

print("\n\n--- Simulación de Intereses Totales vs Plazo ---")
print(f"Capital inicial fijo: {capital_inicial_simple:.2f} euros")
print(f"Tasa de interés anual fija: {tasa_simple:.2f}%")
print("\nDataFrame de resultados (Intereses Totales vs Plazo):")
print(df_intereses_plazo)
# [Image of Interactive Plot of Total Interests vs Mortgage Term]



--- Simulación de Intereses Totales vs Plazo ---
Capital inicial fijo: 200000.00 euros
Tasa de interés anual fija: 3.22%

DataFrame de resultados (Intereses Totales vs Plazo):
   Plazo_en_Años  Plazo_en_Meses  Intereses_Totales
0              1              12        3505.470715
1             10             120       34190.998877
2             15             180       52436.182078
3             20             240       71524.098895
4             25             300       91439.743869
5             30             360      112164.288434
6             35             420      133675.406668
7             40             480      155947.650100


## A tipo fijo con amortizaciones sucesivas

In [ ]:
# Sea cual sea la modificacion que se quiera hacer a las condiciones de una hipoteca, la
# forma de plantearla consiste en considerar que, a efectos teoricos, un cambio a partir de la
# cuota (n+1)-esima puede considerarse como la cancelacion de la hipoteca en la cuota n-esima
# seguida de la apertura de una nueva hipoteca por el capital pendiente de amortizacion Cn
# con las nuevas condiciones deseadas. En particular, si no se desea modificar el numero
# total de cuotas, la nueva hipoteca constara de N − n cuotas

    # Partimos de :
    # cuota_mensual = capital_pendiente * tasa_interes / (12. * 100.) * \
    # 1 / (1 - (1. + tasa_interes / (12. * 100.))**(-plazo))

    # o, equivalentemente,
    # c = C_0 * i / [1 - (1 + i) **(-N)]

    # Ergo
    # N = −log(1 − C_0 * i / c ) / log(1 + i)

In [ ]:
# Se puede repetir este proceso iterativo si se desean simular sucesivas inyecciones de capital
# Seria necesario modificar las variables

# capital_original
# mes_inyeccion
# capital_inyectado
# capital_restante
# intereses_pagados_hasta_inyeccion

# Asi como "plazo_original" en "calculo_hipoteca_con_inyeccion"
# si se redujo el plazo de vencimiento en el paso anterior
# o "cuota_original" en "calculo_hipoteca_con_inyeccion"
# si, por contra, se redujo la cuota mensual
# Ya que, al cambiar las condiciones de una hipoteca, consiste, teoricamente, en
# "empezar" otra desde el mes de modificacion

# N.B.: Si se desean simular los efectos del euribor anualmente, habria que simular una inyeccion de capital nula,
# variando el tipo de interes en "calculo_hipoteca_con_inyeccion", y recordando concatenar las 12 primera filas
# del DataFrame resultante en cada variacion anual

In [ ]:
def simulacion_hipoteca_multiple_inyeccion(capital_inicial, tasa, plazo_inicial,
                                            cuota_inicial, inyecciones):
    """
    Simulación robusta de hipoteca con múltiples inyecciones de capital y tipos de reducción
    definidos para cada inyección a lo largo del tiempo.

    Args:
        capital_inicial (float): Capital inicial del préstamo.
        tasa (float): Tasa de interés anual en porcentaje (ej: 3.5 para 3.5%).
        plazo_inicial (int): Plazo inicial del préstamo en meses.
        cuota_inicial (float): Cuota mensual inicial del préstamo.
        inyecciones (list of dicts): Lista de inyecciones, cada una con:
            - 'mes_inyeccion' (int): Mes en el que se realiza la acción (a partir del mes 1).
            - 'capital_inyectado' (float): Cantidad de capital inyectado (puede ser 0).
            - 'tipo_inyeccion' (str): 'cuota' o 'plazo' - cómo se aplica la inyección,
                                       o None si no se cambia el tipo de reducción en este mes.

    Returns:
        pd.DataFrame: DataFrame con el detalle de la simulación mes a mes.
    """

    # Validaciones iniciales generales
    if capital_inicial <= 0:
        raise ValueError("El capital inicial debe ser positivo")
    if plazo_inicial <= 0:
        raise ValueError("El plazo inicial debe ser positivo")
    if cuota_inicial <= 0:
        raise ValueError("La cuota inicial debe ser positiva")

    # Validaciones de las inyecciones
    if not isinstance(inyecciones, list):
        raise TypeError("Las inyecciones deben ser una lista de diccionarios")
    opcion_reduccion_actual = None # Inicialmente no hay opción definida
    for inyeccion in inyecciones:
        if not isinstance(inyeccion, dict):
            raise TypeError("Cada inyección debe ser un diccionario")
        if 'mes_inyeccion' not in inyeccion:
            raise ValueError("Cada inyección debe tener 'mes_inyeccion'")
        if 'capital_inyectado' not in inyeccion:
            inyeccion['capital_inyectado'] = 0 # Asumir 0 si no se especifica
        if 'tipo_inyeccion' in inyeccion and inyeccion['tipo_inyeccion'] not in ['cuota', 'plazo']:
            raise ValueError("Tipo de inyección debe ser 'cuota' o 'plazo' o None")


    registros = []
    capital_pendiente = capital_inicial
    cuota_actual = cuota_inicial
    plazo_restante = plazo_inicial
    mes_actual = 0

    for mes in range(1, plazo_inicial + 1):
        mes_actual += 1

        # Verificar si hay inyección/acción este mes
        inyeccion_mes = 0
        tipo_inyeccion_mes = None
        for inyeccion in inyecciones:
            if inyeccion['mes_inyeccion'] == mes_actual:
                inyeccion_mes = inyeccion['capital_inyectado']
                tipo_inyeccion_mes = inyeccion['tipo_inyeccion']


        # Aplicar inyección si existe y validar que no supere el capital restante
        if inyeccion_mes > 0:
            if inyeccion_mes > capital_pendiente:
                raise ValueError(f"Inyección en el mes {mes_actual} supera el capital pendiente.")
            capital_pendiente -= inyeccion_mes


        if capital_pendiente <= 0:
            registros.append({
                'Mes': mes_actual,
                'Capital_pendiente': 0,
                'Cuota_mensual': 0,
                'Intereses_mensuales': 0,
                'Amortizacion_mensual': 0,
                'Inyeccion_capital': inyeccion_mes,
                'Tipo_Reduccion': tipo_inyeccion_mes if tipo_inyeccion_mes else opcion_reduccion_actual
            })
            break  # Finalizar si ya se pagó todo


        interes = intereses_mensuales(capital_pendiente, tasa)
        amortizacion = min(cuota_actual - interes, capital_pendiente)  # No sobrepagar

        registros.append({
            'Mes': mes_actual,
            'Capital_pendiente': capital_pendiente,
            'Cuota_mensual': cuota_actual,
            'Intereses_mensuales': interes,
            'Amortizacion_mensual': amortizacion,
            'Inyeccion_capital': inyeccion_mes,
            'Tipo_Reduccion': tipo_inyeccion_mes if tipo_inyeccion_mes else opcion_reduccion_actual
        })

        capital_pendiente -= amortizacion

        # Recalcular cuota o plazo si hubo inyección o cambio de tipo y aún queda préstamo
        if (inyeccion_mes > 0 or tipo_inyeccion_mes) and capital_pendiente > 0:

            if tipo_inyeccion_mes: # Si se especifica un tipo en este mes, se actualiza la opción actual
                opcion_reduccion_actual = tipo_inyeccion_mes

            nuevo_capital = capital_pendiente

            if opcion_reduccion_actual == 'cuota':
                plazo_restante_recalculo = max(plazo_inicial - mes_actual, 1) # Usar plazo inicial como referencia
                cuota_actual = cuota_mensual(nuevo_capital, tasa, plazo_restante_recalculo)

            elif opcion_reduccion_actual == 'plazo':
                cuota_actual_recalculo = cuota_actual # Mantenemos la cuota original
                plazo_restante = calcular_plazo(nuevo_capital, tasa, cuota_actual_recalculo)
                plazo_inicial = mes_actual + plazo_restante # Ajustamos plazo inicial para futuras inyecciones



    return pd.DataFrame(registros)



def calcular_ahorro_intereses_multiple_inyeccion(capital_inicial, tasa, plazo_inicial,
                                                 cuota_inicial, inyecciones): # Eliminamos opcion_reduccion_base
    """
    Calcula el ahorro total de intereses al realizar múltiples inyecciones de capital
    Y TIPOS de reducción, comparado con una simulación base: la hipoteca original SIN inyecciones.

    Args:
        capital_inicial (float): Capital inicial del préstamo.
        tasa (float): Tasa de interés anual en porcentaje.
        plazo_inicial (int): Plazo inicial del préstamo en meses.
        cuota_inicial (float): Cuota mensual inicial del préstamo.
        inyecciones (list of dicts): Lista de inyecciones (ver docstring de simulacion_hipoteca_multiple_inyeccion).

    Returns:
        float: Ahorro total de intereses.
    """

    # Simulación con inyecciones y tipos de reducción definidos
    df_con_inyecciones = simulacion_hipoteca_multiple_inyeccion(
        capital_inicial, tasa, plazo_inicial, cuota_inicial, inyecciones)
    intereses_con_inyecciones = df_con_inyecciones['Intereses_mensuales'].sum()

    # Simulación base SIN inyecciones (hipoteca original)
    df_sin_inyecciones = simulacion_hipoteca_multiple_inyeccion(
        capital_inicial, tasa, plazo_inicial, cuota_inicial, inyecciones=[]) # Lista vacía = sin inyecciones
    intereses_sin_inyecciones = df_sin_inyecciones['Intereses_mensuales'].sum()

    ahorro_intereses = intereses_sin_inyecciones - intereses_con_inyecciones
    return ahorro_intereses, intereses_sin_inyecciones, intereses_con_inyecciones


def plot_comparacion(df_hipoteca_original, df_hipoteca_con_inyecciones):

    # --- Plotly Interactive Plot ---
    fig = go.Figure()


    # Original (sin inyecciones)
    fig.add_trace(go.Scatter(x=df_hipoteca_original['Mes'], # Usamos df_hipoteca_original
                              y=df_hipoteca_original['Cuota_mensual'],
                              mode='lines',
                              line=dict(color='blue'),
                              name='Cuota Original'))

    fig.add_trace(go.Scatter(x=df_hipoteca_original['Mes'], # Usamos df_hipoteca_original
                              y=df_hipoteca_original['Amortizacion_mensual'],
                              mode='lines',
                              line=dict(color='red'),
                              name='Amortización Original'))

    fig.add_trace(go.Scatter(x=df_hipoteca_original['Mes'], # Usamos df_hipoteca_original
                              y=df_hipoteca_original['Intereses_mensuales'],
                              mode='lines',
                              line=dict(color='black'),
                              name='Intereses Originales'))


    # Hipoteca con Inyecciones Combinadas
    fig.add_trace(go.Scatter(x=df_hipoteca_con_inyecciones['Mes'], # Usamos df_hipoteca_con_inyecciones
                              y=df_hipoteca_con_inyecciones['Cuota_mensual'],
                              mode='lines',
                              line=dict(color='blue', dash='dot'),
                              showlegend=False)) # Eliminar leyenda duplicada

    fig.add_trace(go.Scatter(x=df_hipoteca_con_inyecciones['Mes'], # Usamos df_hipoteca_con_inyecciones
                              y=df_hipoteca_con_inyecciones['Amortizacion_mensual'],
                              mode='lines',
                              line=dict(color='red', dash='dot'),
                              showlegend=False)) # Eliminar leyenda duplicada

    fig.add_trace(go.Scatter(x=df_hipoteca_con_inyecciones['Mes'], # Usamos df_hipoteca_con_inyecciones
                              y=df_hipoteca_con_inyecciones['Intereses_mensuales'],
                              mode='lines',
                              line=dict(color='black', dash='dot'),
                              showlegend=False,
                              name='Con Inyecciones')) # Cambiamos leyenda a "Con Inyecciones"



    fig.update_layout(
        xaxis_title="Mes",
        yaxis_title="Importe (euros)",
        font_family="Courier New",
        font_color="black",
        title_font_family="Times New Roman",
        title_font_color="black",
        legend_title_font_color="black",
        title='Comparativa Hipoteca Original vs. Hipoteca con Amortizaciones Anticipadas' # Título más descriptivo
    )

    fig.update_layout(legend=dict(
        yanchor="bottom", #  La leyenda ahora se ancla en la parte inferior
        y=0.02,         # Ajustamos la posición vertical ligeramente desde la parte inferior
        xanchor="left",
        x=0.01,
        bgcolor="white",
        bordercolor="black",
        borderwidth=1,
        font_family="Arial",
    ))

    fig.show()


### Diferentes escenarios

In [ ]:
# Ejemplo de uso con tipos de inyección combinados
capital_inicial = 150000
tasa_anual = 3.5
plazo_inicial_meses = 30 * 12 # 30 años
cuota_inicial_calculada = cuota_mensual(capital_inicial, tasa_anual, plazo_inicial_meses)


# Definir inyecciones con tipo de inyección
inyecciones_ejemplo_combinado = [
    #{'mes_inyeccion': 60, 'capital_inyectado': 10000, 'tipo_inyeccion': 'plazo'},
    #{'mes_inyeccion': 120, 'capital_inyectado': 10000, 'tipo_inyeccion': 'plazo'},
    #{'mes_inyeccion': 180, 'capital_inyectado': 10000, 'tipo_inyeccion': 'plazo'}
]


# Simulación SIN inyecciones (hipoteca original)
df_hipoteca_original = simulacion_hipoteca_multiple_inyeccion( # Cambiamos nombre variable a original
    capital_inicial, tasa_anual, plazo_inicial_meses, cuota_inicial_calculada,
    inyecciones=[]) # Lista de inyecciones vacía = sin inyecciones


# Simulación con inyecciones combinadas
df_hipoteca_con_inyecciones = simulacion_hipoteca_multiple_inyeccion( # Cambiamos nombre variable a con_inyecciones
    capital_inicial, tasa_anual, plazo_inicial_meses, cuota_inicial_calculada,
    inyecciones=inyecciones_ejemplo_combinado)


# --- Cálculo del ahorro de intereses (comparado con hipoteca original) ---
ahorro_total_vs_original, intereses_sin_inyecciones, intereses_con_inyecciones = calcular_ahorro_intereses_multiple_inyeccion( # Calculamos el ahorro vs original
    capital_inicial, tasa_anual, plazo_inicial_meses, cuota_inicial_calculada,
    inyecciones=inyecciones_ejemplo_combinado)


print("Condiciones iniciales:") # Encabezado más descriptivo
print(f"  Capital inicial: {capital_inicial:,.2f} €") # Imprimir capital_inicial correctamente
print(f"  Tasa anual: {tasa_anual:.2f} %") # Imprimir tasa_anual correctamente
print(f"  Plazo inicial: {plazo_inicial_meses // 12} años") # Imprimir plazo en años

print("Inyecciones:") # Encabezado para inyecciones
for inyeccion in inyecciones_ejemplo_combinado: # Iterar sobre la lista de inyecciones
    print(f"  Mes: {inyeccion['mes_inyeccion']}, Capital: {inyeccion['capital_inyectado']:,.2f} €, Tipo: {inyeccion['tipo_inyeccion']}") # Imprimir cada inyección formateada


print("Resultados:") # Encabezado
print(f"Ahorro total de intereses (vs. Hipoteca Original): {ahorro_total_vs_original:,.2f} €") # Imprimimos el ahorro vs original
print(f"Intereses sin inyecciones: {intereses_sin_inyecciones:,.2f} €") # Imprimimos intereses sin inyecciones
print(f"Intereses con inyecciones: {intereses_con_inyecciones:,.2f} €")

plot_comparacion(df_hipoteca_original, df_hipoteca_con_inyecciones)


Condiciones iniciales:
  Capital inicial: 150,000.00 €
  Tasa anual: 3.50 %
  Plazo inicial: 30 años
Inyecciones:
Resultados:
Ahorro total de intereses (vs. Hipoteca Original): 0.00 €
Intereses sin inyecciones: 92,484.13 €
Intereses con inyecciones: 92,484.13 €


In [ ]:
# Ejemplo de uso con tipos de inyección combinados
capital_inicial = 149765
tasa_anual = 3.5
plazo_inicial_meses = 30 * 12 # 30 años
cuota_inicial_calculada = cuota_mensual(capital_inicial, tasa_anual, plazo_inicial_meses)


# Definir inyecciones con tipo de inyección
inyecciones_ejemplo_combinado = [
    {'mes_inyeccion': 60, 'capital_inyectado': 10000, 'tipo_inyeccion': 'cuota'},
    {'mes_inyeccion': 120, 'capital_inyectado': 10000, 'tipo_inyeccion': 'cuota'},
    {'mes_inyeccion': 180, 'capital_inyectado': 10000, 'tipo_inyeccion': 'cuota'}
]


# Simulación SIN inyecciones (hipoteca original)
df_hipoteca_original = simulacion_hipoteca_multiple_inyeccion( # Cambiamos nombre variable a original
    capital_inicial, tasa_anual, plazo_inicial_meses, cuota_inicial_calculada,
    inyecciones=[]) # Lista de inyecciones vacía = sin inyecciones


# Simulación con inyecciones combinadas
df_hipoteca_con_inyecciones = simulacion_hipoteca_multiple_inyeccion( # Cambiamos nombre variable a con_inyecciones
    capital_inicial, tasa_anual, plazo_inicial_meses, cuota_inicial_calculada,
    inyecciones=inyecciones_ejemplo_combinado)


# --- Cálculo del ahorro de intereses (comparado con hipoteca original) ---
ahorro_total_vs_original, intereses_sin_inyecciones, intereses_con_inyecciones = calcular_ahorro_intereses_multiple_inyeccion( # Calculamos el ahorro vs original
    capital_inicial, tasa_anual, plazo_inicial_meses, cuota_inicial_calculada,
    inyecciones=inyecciones_ejemplo_combinado)


print("Condiciones iniciales:") # Encabezado más descriptivo
print(f"  Capital inicial: {capital_inicial:,.2f} €") # Imprimir capital_inicial correctamente
print(f"  Tasa anual: {tasa_anual:.2f} %") # Imprimir tasa_anual correctamente
print(f"  Plazo inicial: {plazo_inicial_meses // 12} años") # Imprimir plazo en años

print("Inyecciones:") # Encabezado para inyecciones
for inyeccion in inyecciones_ejemplo_combinado: # Iterar sobre la lista de inyecciones
    print(f"  Mes: {inyeccion['mes_inyeccion']}, Capital: {inyeccion['capital_inyectado']:,.2f} €, Tipo: {inyeccion['tipo_inyeccion']}") # Imprimir cada inyección formateada


print("Resultados:") # Encabezado
print(f"Ahorro total de intereses (vs. Hipoteca Original): {ahorro_total_vs_original:,.2f} €") # Imprimimos el ahorro vs original
print(f"Intereses sin inyecciones: {intereses_sin_inyecciones:,.2f} €") # Imprimimos intereses sin inyecciones
print(f"Intereses con inyecciones: {intereses_con_inyecciones:,.2f} €")

plot_comparacion(df_hipoteca_original, df_hipoteca_con_inyecciones)


Condiciones iniciales:
  Capital inicial: 149,765.00 €
  Tasa anual: 3.50 %
  Plazo inicial: 30 años
Inyecciones:
  Mes: 60, Capital: 10,000.00 €, Tipo: cuota
  Mes: 120, Capital: 10,000.00 €, Tipo: cuota
  Mes: 180, Capital: 10,000.00 €, Tipo: cuota
Resultados:
Ahorro total de intereses (vs. Hipoteca Original): 11,927.56 €
Intereses sin inyecciones: 92,339.24 €
Intereses con inyecciones: 80,411.68 €


### Checks

In [ ]:
df_hipoteca_original["Amortizacion_mensual"].sum(), df_hipoteca_con_inyecciones["Amortizacion_mensual"].sum() + 30000, capital_inicial

(149764.99999999892, 149764.99999999985, 149765)

## A tipo variable

In [ ]:
def simulacion_hipoteca_variable(capital_inicial, spread, plazo_inicial,
                                 cuota_inicial, euribor_anual_values):
    """
    Simulación de hipoteca a tipo variable referenciada al Euribor a 12 meses.

    Args:
        capital_inicial (float): Capital inicial del préstamo.
        spread (float): Diferencial (spread) sobre el Euribor en porcentaje (ej: 1.5 para Euribor + 1.5%).
        plazo_inicial (int): Plazo inicial del préstamo en meses.
        cuota_inicial (float): Cuota mensual inicial calculada con el primer valor de Euribor.
        euribor_anual_values (list): Lista de valores anuales del Euribor a 12 meses en porcentaje.
                                     Se asume que la revisión de tipo se hace anualmente.
                                     Ej: [0.5, 1.0, 1.5, ...]  El primer valor aplica para el año 1, el segundo para el año 2, etc.

    Returns:
        pd.DataFrame: DataFrame con el detalle de la simulación mes a mes.
    """

    # Validaciones iniciales generales
    if capital_inicial <= 0:
        raise ValueError("El capital inicial debe ser positivo")
    if plazo_inicial <= 0:
        raise ValueError("El plazo inicial debe ser positivo")
    if cuota_inicial <= 0:
        raise ValueError("La cuota inicial debe ser positiva")
    if not isinstance(euribor_anual_values, list) or not euribor_anual_values:
        raise ValueError("Se debe proporcionar una lista de valores de Euribor anuales.")
    if not all(isinstance(val, (int, float)) for val in euribor_anual_values):
        raise TypeError("Los valores de Euribor deben ser numéricos.")


    registros = []
    capital_pendiente = capital_inicial
    cuota_mensual_fija = cuota_inicial  # Inicialmente fijada, puede recalcularse
    mes_actual = 0
    plazo_restante = plazo_inicial # Plazo restante que se irá actualizando
    year_actual = 0 # Para controlar el cambio de año y aplicar el Euribor correspondiente

    for mes in range(1, plazo_inicial + 1):
        mes_actual += 1
        year_in_simulation = math.ceil(mes_actual / 12) # Calcula el año en la simulación

        # Actualizar tipo de interés anualmente según los valores de Euribor proporcionados
        if year_in_simulation > year_actual and year_in_simulation <= len(euribor_anual_values):
            year_actual = year_in_simulation
            euribor_anual_actual = euribor_anual_values[year_actual-1] # Índice base 0
            tasa_anual_actual = euribor_anual_actual + spread # Euribor + diferencial
            cuota_mensual_fija = cuota_mensual(capital_pendiente, tasa_anual_actual, plazo_restante) # Recalcular cuota con nuevo tipo y capital pendiente
            print(f"Año {year_actual}, Euribor: {euribor_anual_actual}%, Tasa Anual Actual: {tasa_anual_actual:.2f}%, Nueva Cuota: {cuota_mensual_fija:.2f}")


        elif year_in_simulation > len(euribor_anual_values):
            tasa_anual_actual = tasa_anual_actual # Mantiene la última tasa si se acaban los valores de Euribor

        elif year_actual == 0 and year_in_simulation == 1: # Primer año
             euribor_anual_actual = euribor_anual_values[0] # Primer valor de Euribor para el inicio
             tasa_anual_actual = euribor_anual_actual + spread


        if capital_pendiente <= 0:
            registros.append({
                'Mes': mes_actual,
                'Año_Simulacion': year_in_simulation,
                'Euribor_Anual': euribor_anual_actual if 'euribor_anual_actual' in locals() else 0, # Asegurar que exista en el primer mes
                'Tasa_Anual': tasa_anual_actual if 'tasa_anual_actual' in locals() else 0, # Asegurar que exista en el primer mes
                'Capital_pendiente': 0,
                'Cuota_mensual': 0,
                'Intereses_mensuales': 0,
                'Amortizacion_mensual': 0
            })
            break  # Finalizar si ya se pagó todo

        interes = intereses_mensuales(capital_pendiente, tasa_anual_actual) # Usar la tasa anual actual
        amortizacion = min(cuota_mensual_fija - interes, capital_pendiente)  # No sobrepagar

        registros.append({
            'Mes': mes_actual,
            'Año_Simulacion': year_in_simulation,
            'Euribor_Anual': euribor_anual_actual if 'euribor_anual_actual' in locals() else 0, # Asegurar que exista en el primer mes
            'Tasa_Anual': tasa_anual_actual if 'tasa_anual_actual' in locals() else 0, # Asegurar que exista en el primer mes
            'Capital_pendiente': capital_pendiente,
            'Cuota_mensual': cuota_mensual_fija,
            'Intereses_mensuales': interes,
            'Amortizacion_mensual': amortizacion
        })

        capital_pendiente -= amortizacion
        plazo_restante -=1 # Reducir el plazo restante cada mes


    return pd.DataFrame(registros)


def calcular_intereses_totales_variable(capital_inicial, spread, plazo_inicial,
                                        cuota_inicial, euribor_anual_values):
    """
    Calcula los intereses totales de una hipoteca variable.

    Args:
        capital_inicial (float): Capital inicial del préstamo.
        spread (float): Diferencial sobre el Euribor.
        plazo_inicial (int): Plazo inicial del préstamo en meses.
        cuota_inicial (float): Cuota mensual inicial.
        euribor_anual_values (list): Lista de valores anuales del Euribor.

    Returns:
        float: Intereses totales pagados durante la vida del préstamo.
    """
    df_simulacion = simulacion_hipoteca_variable(capital_inicial, spread, plazo_inicial,
                                                   cuota_inicial, euribor_anual_values)
    intereses_totales = df_simulacion['Intereses_mensuales'].sum()
    return intereses_totales


def plot_hipoteca_variable(df_hipoteca_variable):

    # --- Plotly Interactive Plot ---
    fig = go.Figure()


    # Hipoteca Variable
    fig.add_trace(go.Scatter(x=df_hipoteca_variable['Mes'],
                            y=df_hipoteca_variable['Cuota_mensual'],
                            mode='lines',
                            line=dict(color='blue'),
                            name='Cuota Mensual'))

    fig.add_trace(go.Scatter(x=df_hipoteca_variable['Mes'],
                            y=df_hipoteca_variable['Amortizacion_mensual'],
                            mode='lines',
                            line=dict(color='red'),
                            name='Amortización Mensual'))

    fig.add_trace(go.Scatter(x=df_hipoteca_variable['Mes'],
                            y=df_hipoteca_variable['Intereses_mensuales'],
                            mode='lines',
                            line=dict(color='black'),
                            name='Intereses Mensuales'))


    fig.update_layout(
        xaxis_title="Mes",
        yaxis_title="Importe (euros)",
        font_family="Courier New",
        font_color="black",
        title_font_family="Times New Roman",
        title_font_color="black",
        legend_title_font_color="black",
        title='Simulación de Hipoteca Variable (Euribor + Spread)'
    )

    fig.update_layout(legend=dict(
        yanchor="bottom", #  La leyenda ahora se ancla en la parte inferior
        y=0.02,        # Ajustamos la posición vertical ligeramente desde la parte inferior
        xanchor="left",
        x=0.01,
        bgcolor="white",
        bordercolor="black",
        borderwidth=1,
        font_family="Arial",
    ))

    fig.show()



if __name__ == '__main__':
    # Ejemplo de uso de la simulación de hipoteca variable:
    capital_inicial_variable = 200000
    spread_variable = 1.5 # Diferencial del 1.5%
    plazo_inicial_variable = 30 * 12 # 30 años en meses

    # Escenario de Euribor a 12 meses (ejemplo - esto debe ser sustituido por datos reales o previsiones)
    euribor_anual_values_ejemplo = [1] #[0.5, 0.7, 1.0, 1.2, 1.5, 1.3, 1.0, 0.8, 1.0, 1.2,
                                     #1.5, 1.8, 2.0, 2.2, 2.5, 2.3, 2.0, 1.8, 2.0, 2.2,
                                     #2.5, 2.8, 3.0, 3.2, 3.5, 3.3, 3.0, 2.8, 3.0, 3.2] # Euribor para 30 años

    # Calcular la cuota inicial con el primer valor de Euribor
    tasa_anual_inicial = euribor_anual_values_ejemplo[0] + spread_variable
    cuota_inicial_variable = cuota_mensual(capital_inicial_variable, tasa_anual_inicial, plazo_inicial_variable)


    df_hipoteca_variable = simulacion_hipoteca_variable(capital_inicial_variable, spread_variable, plazo_inicial_variable,
                                                         cuota_inicial_variable, euribor_anual_values_ejemplo)
    intereses_totales_variable = calcular_intereses_totales_variable(capital_inicial_variable, spread_variable, plazo_inicial_variable,
                                                                     cuota_inicial_variable, euribor_anual_values_ejemplo)


    print("\n--- Simulación de Hipoteca Variable ---")
    print(f"Capital Inicial: {capital_inicial_variable:,.2f} euros")
    print(f"Spread: {spread_variable} %")
    print(f"Plazo Inicial: {plazo_inicial_variable // 12} años")

    print(f"Intereses totales a pagar (hipoteca variable): {intereses_totales_variable:.2f} euros")
    print("\nDataFrame de la simulación de hipoteca variable:")
    #print(df_hipoteca_variable)

    plot_hipoteca_variable(df_hipoteca_variable)

Año 1, Euribor: 1%, Tasa Anual Actual: 2.50%, Nueva Cuota: 790.24
Año 1, Euribor: 1%, Tasa Anual Actual: 2.50%, Nueva Cuota: 790.24

--- Simulación de Hipoteca Variable ---
Capital Inicial: 200,000.00 euros
Spread: 1.5 %
Plazo Inicial: 30 años
Intereses totales a pagar (hipoteca variable): 84487.05 euros

DataFrame de la simulación de hipoteca variable:


## A tipo variable con amortizaciones sucesivas

In [ ]:
def simulacion_hipoteca_variable_inyeccion(capital_inicial, spread, plazo_inicial,
                                             cuota_inicial, euribor_anual_values, inyecciones):
    """
    Simulación de hipoteca a tipo variable con amortizaciones anticipadas.

    Args:
        capital_inicial (float): Capital inicial del préstamo.
        spread (float): Diferencial (spread) sobre el Euribor en porcentaje.
        plazo_inicial (int): Plazo inicial del préstamo en meses.
        cuota_inicial (float): Cuota mensual inicial (calculada con el primer Euribor).
        euribor_anual_values (list): Lista de valores anuales del Euribor a 12 meses en porcentaje.
        inyecciones (list of dicts): Lista de inyecciones (ver docstring de simulacion_hipoteca_multiple_inyeccion).

    Returns:
        pd.DataFrame: DataFrame con el detalle de la simulación mes a mes.
    """

    # Validaciones iniciales generales (añadidas las de inyecciones)
    if capital_inicial <= 0:
        raise ValueError("El capital inicial debe ser positivo")
    if plazo_inicial <= 0:
        raise ValueError("El plazo inicial debe ser positivo")
    if cuota_inicial <= 0:
        raise ValueError("La cuota inicial debe ser positiva")
    if not isinstance(euribor_anual_values, list) or not euribor_anual_values:
        raise ValueError("Se debe proporcionar una lista de valores de Euribor anuales.")
    if not all(isinstance(val, (int, float)) for val in euribor_anual_values):
        raise TypeError("Los valores de Euribor deben ser numéricos.")
    if not isinstance(inyecciones, list):
        raise TypeError("Las inyecciones deben ser una lista de diccionarios")
    opcion_reduccion_actual = None # Inicialmente no hay opción definida
    for inyeccion in inyecciones:
        if not isinstance(inyeccion, dict):
            raise TypeError("Cada inyección debe ser un diccionario")
        if 'mes_inyeccion' not in inyeccion:
            raise ValueError("Cada inyección debe tener 'mes_inyeccion'")
        if 'capital_inyectado' not in inyeccion:
            inyeccion['capital_inyectado'] = 0 # Asumir 0 si no se especifica
        if 'tipo_inyeccion' in inyeccion and inyeccion['tipo_inyeccion'] not in ['cuota', 'plazo']:
            raise ValueError("Tipo de inyección debe ser 'cuota' o 'plazo' o None")


    registros = []
    capital_pendiente = capital_inicial
    cuota_mensual_fija = cuota_inicial  # Inicialmente fijada, puede recalcularse
    mes_actual = 0
    plazo_restante = plazo_inicial # Plazo restante que se irá actualizando
    year_actual = 0 # Para controlar el cambio de año y aplicar el Euribor correspondiente


    for mes in range(1, plazo_inicial + 1):
        mes_actual += 1
        year_in_simulation = math.ceil(mes_actual / 12) # Calcula el año en la simulación

        # Actualizar tipo de interés anualmente según los valores de Euribor proporcionados
        if year_in_simulation > year_actual and year_in_simulation <= len(euribor_anual_values):
            year_actual = year_in_simulation
            euribor_anual_actual = euribor_anual_values[year_actual-1] # Índice base 0
            tasa_anual_actual = euribor_anual_actual + spread # Euribor + diferencial
            cuota_mensual_fija = cuota_mensual(capital_pendiente, tasa_anual_actual, plazo_restante) # Recalcular cuota con nuevo tipo y capital pendiente
            #print(f"Año {year_actual}, Euribor: {euribor_anual_actual}%, Tasa Anual Actual: {tasa_anual_actual:.2f}%, Nueva Cuota: {cuota_mensual_fija:.2f}")


        elif year_in_simulation > len(euribor_anual_values):
            tasa_anual_actual = tasa_anual_actual # Mantiene la última tasa si se acaban los valores de Euribor

        elif year_actual == 0 and year_in_simulation == 1: # Primer año
             euribor_anual_actual = euribor_anual_values[0] # Primer valor de Euribor para el inicio
             tasa_anual_actual = euribor_anual_actual + spread


        # Verificar si hay inyección/acción este mes (MISMO CÓDIGO QUE EN SIMULACION MULTIPLE INYECCION)
        inyeccion_mes = 0
        tipo_inyeccion_mes = None
        for inyeccion in inyecciones:
            if inyeccion['mes_inyeccion'] == mes_actual:
                inyeccion_mes = inyeccion['capital_inyectado']
                tipo_inyeccion_mes = inyeccion['tipo_inyeccion']


        # Aplicar inyección si existe y validar que no supere el capital restante (MISMO CÓDIGO QUE EN SIMULACION MULTIPLE INYECCION)
        if inyeccion_mes > 0:
            if inyeccion_mes > capital_pendiente:
                raise ValueError(f"Inyección en el mes {mes_actual} supera el capital pendiente.")
            capital_pendiente -= inyeccion_mes


        if capital_pendiente <= 0:
            registros.append({
                'Mes': mes_actual,
                'Año_Simulacion': year_in_simulation,
                'Euribor_Anual': euribor_anual_actual if 'euribor_anual_actual' in locals() else 0,
                'Tasa_Anual': tasa_anual_actual if 'tasa_anual_actual' in locals() else 0,
                'Capital_pendiente': 0,
                'Cuota_mensual': 0,
                'Intereses_mensuales': 0,
                'Amortizacion_mensual': 0,
                'Inyeccion_capital': inyeccion_mes,
                'Tipo_Reduccion': tipo_inyeccion_mes if tipo_inyeccion_mes else opcion_reduccion_actual
            })
            break  # Finalizar si ya se pagó todo


        interes = intereses_mensuales(capital_pendiente, tasa_anual_actual) # Usar la tasa anual actual
        amortizacion = min(cuota_mensual_fija - interes, capital_pendiente)  # No sobrepagar

        registros.append({
            'Mes': mes_actual,
            'Año_Simulacion': year_in_simulation,
            'Euribor_Anual': euribor_anual_actual if 'euribor_anual_actual' in locals() else 0,
            'Tasa_Anual': tasa_anual_actual if 'tasa_anual_actual' in locals() else 0,
            'Capital_pendiente': capital_pendiente,
            'Cuota_mensual': cuota_mensual_fija,
            'Intereses_mensuales': interes,
            'Amortizacion_mensual': amortizacion,
            'Inyeccion_capital': inyeccion_mes,
            'Tipo_Reduccion': tipo_inyeccion_mes if tipo_inyeccion_mes else opcion_reduccion_actual
        })

        capital_pendiente -= amortizacion
        plazo_restante -=1 # Reducir el plazo restante cada mes


        # Recalcular cuota o plazo si hubo inyección o cambio de tipo y aún queda préstamo (MISMO CÓDIGO QUE EN SIMULACION MULTIPLE INYECCION)
        if (inyeccion_mes > 0 or tipo_inyeccion_mes) and capital_pendiente > 0:

            if tipo_inyeccion_mes: # Si se especifica un tipo en este mes, se actualiza la opción actual
                opcion_reduccion_actual = tipo_inyeccion_mes

            nuevo_capital = capital_pendiente

            if opcion_reduccion_actual == 'cuota':
                plazo_restante_recalculo = max(plazo_inicial - mes_actual, 1) # Usar plazo inicial como referencia
                cuota_mensual_fija = cuota_mensual(nuevo_capital, tasa_anual_actual, plazo_restante_recalculo) # Usar la tasa anual ACTUAL

            elif opcion_reduccion_actual == 'plazo':
                cuota_actual_recalculo = cuota_mensual_fija # Mantenemos la cuota actual RECALCULADA previamente por el euribor
                plazo_restante = calcular_plazo(nuevo_capital, tasa_anual_actual, cuota_actual_recalculo) # Usar la tasa anual ACTUAL
                plazo_inicial = mes_actual + plazo_restante # Ajustamos plazo inicial para futuras inyecciones



    return pd.DataFrame(registros)


def calcular_ahorro_intereses_variable_inyeccion(capital_inicial, spread, plazo_inicial,
                                                        cuota_inicial, euribor_anual_values, inyecciones):
    """
    Calcula el ahorro total de intereses en hipoteca variable con inyecciones.
    Compara con una simulación base de hipoteca variable SIN inyecciones.

    Args:
        capital_inicial (float): Capital inicial del préstamo.
        spread (float): Diferencial sobre el Euribor.
        plazo_inicial (int): Plazo inicial del préstamo en meses.
        cuota_inicial (float): Cuota mensual inicial.
        euribor_anual_values (list): Lista de valores anuales del Euribor.
        inyecciones (list of dicts): Lista de inyecciones (ver docstring de simulacion_hipoteca_variable_inyeccion).

    Returns:
        float: Ahorro total de intereses.
    """

    # Simulación con inyecciones y tipos de reducción definidos
    df_con_inyecciones = simulacion_hipoteca_variable_inyeccion(
        capital_inicial, spread, plazo_inicial, cuota_inicial, euribor_anual_values, inyecciones)
    intereses_con_inyecciones = df_con_inyecciones['Intereses_mensuales'].sum()

    # Simulación base SIN inyecciones (hipoteca variable original)
    df_sin_inyecciones = simulacion_hipoteca_variable_inyeccion(
        capital_inicial, spread, plazo_inicial, cuota_inicial, euribor_anual_values, inyecciones=[]) # Lista vacía = sin inyecciones
    intereses_sin_inyecciones = df_sin_inyecciones['Intereses_mensuales'].sum()

    ahorro_intereses = intereses_sin_inyecciones - intereses_con_inyecciones
    return ahorro_intereses, intereses_sin_inyecciones, intereses_con_inyecciones


def plot_comparacion_variable(df_hipoteca_original, df_hipoteca_con_inyecciones):

    # --- Plotly Interactive Plot ---
    fig = go.Figure()


    # Original (sin inyecciones)
    fig.add_trace(go.Scatter(x=df_hipoteca_original['Mes'], # Usamos df_hipoteca_original
                            y=df_hipoteca_original['Cuota_mensual'],
                            mode='lines',
                            line=dict(color='blue'),
                            name='Cuota Original (Variable)'))

    fig.add_trace(go.Scatter(x=df_hipoteca_original['Mes'], # Usamos df_hipoteca_original
                            y=df_hipoteca_original['Amortizacion_mensual'],
                            mode='lines',
                            line=dict(color='red'),
                            name='Amortización Original (Variable)'))

    fig.add_trace(go.Scatter(x=df_hipoteca_original['Mes'], # Usamos df_hipoteca_original
                            y=df_hipoteca_original['Intereses_mensuales'],
                            mode='lines',
                            line=dict(color='black'),
                            name='Intereses Originales (Variable)'))


    # Hipoteca con Inyecciones Combinadas
    fig.add_trace(go.Scatter(x=df_hipoteca_con_inyecciones['Mes'], # Usamos df_hipoteca_con_inyecciones
                            y=df_hipoteca_con_inyecciones['Cuota_mensual'],
                            mode='lines',
                            line=dict(color='blue', dash='dot'),
                            showlegend=False)) # Eliminar leyenda duplicada

    fig.add_trace(go.Scatter(x=df_hipoteca_con_inyecciones['Mes'], # Usamos df_hipoteca_con_inyecciones
                            y=df_hipoteca_con_inyecciones['Amortizacion_mensual'],
                            mode='lines',
                            line=dict(color='red', dash='dot'),
                            showlegend=False)) # Eliminar leyenda duplicada

    fig.add_trace(go.Scatter(x=df_hipoteca_con_inyecciones['Mes'], # Usamos df_hipoteca_con_inyecciones
                            y=df_hipoteca_con_inyecciones['Intereses_mensuales'],
                            mode='lines',
                            line=dict(color='black', dash='dot'),
                            showlegend=False,
                            name='Con Inyecciones (Variable)')) # Cambiamos leyenda a "Con Inyecciones"



    fig.update_layout(
        xaxis_title="Mes",
        yaxis_title="Importe (euros)",
        font_family="Courier New",
        font_color="black",
        title_font_family="Times New Roman",
        title_font_color="black",
        legend_title_font_color="black",
        title='Comparativa Hipoteca Variable Original vs. Hipoteca Variable con Amortizaciones Anticipadas' # Título más descriptivo
    )

    fig.update_layout(legend=dict(
        yanchor="bottom", #  La leyenda ahora se ancla en la parte inferior
        y=0.02,        # Ajustamos la posición vertical ligeramente desde la parte inferior
        xanchor="left",
        x=0.01,
        bgcolor="white",
        bordercolor="black",
        borderwidth=1,
        font_family="Arial",
    ))

    fig.show()


if __name__ == '__main__':
    # Ejemplo de uso de la simulación de hipoteca variable CON inyecciones:
    capital_inicial_variable_inyeccion = 200000
    spread_variable_inyeccion = 1.5 # Diferencial del 1.5%
    plazo_inicial_variable_inyeccion = 30 * 12 # 30 años en meses

    # Escenario de Euribor a 12 meses (ejemplo)
    euribor_anual_values_ejemplo_inyeccion = [0.5, 0.7, 1.0, 1.2, 1.5, 1.3, 1.0, 0.8, 1.0, 1.2,
                                                 1.5, 1.8, 2.0, 2.2, 2.5, 2.3, 2.0, 1.8, 2.0, 2.2,
                                                 2.5, 2.8, 3.0, 3.2, 3.5, 3.3, 3.0, 2.8, 3.0, 3.2] # Euribor para 30 años


    # Inyecciones de capital (ejemplo)
    inyecciones_variable = [
        {'mes_inyeccion': 60, 'capital_inyectado': 10000, 'tipo_inyeccion': 'cuota'}, # Inyección al final del primer año, reduce cuota
        #{'mes_inyeccion': 24, 'capital_inyectado': 15000, 'tipo_inyeccion': 'plazo'}, # Inyección al final del segundo año, reduce plazo
        #{'mes_inyeccion': 60, 'capital_inyectado': 5000, 'tipo_inyeccion': None},      # Inyección al final del quinto año, sin cambiar tipo de reducción
        #{'mes_inyeccion': 120, 'capital_inyectado': 20000, 'tipo_inyeccion': 'cuota'} # Inyección al final del décimo año, reduce cuota
    ]

    # Calcular la cuota inicial con el primer valor de Euribor (igual que en la simulación variable simple)
    tasa_anual_inicial_variable_inyeccion = euribor_anual_values_ejemplo_inyeccion[0] + spread_variable_inyeccion
    cuota_inicial_variable_inyeccion = cuota_mensual(capital_inicial_variable_inyeccion, tasa_anual_inicial_variable_inyeccion, plazo_inicial_variable_inyeccion)


    df_hipoteca_variable_con_inyecciones = simulacion_hipoteca_variable_inyeccion(
        capital_inicial_variable_inyeccion, spread_variable_inyeccion, plazo_inicial_variable_inyeccion,
        cuota_inicial_variable_inyeccion, euribor_anual_values_ejemplo_inyeccion, inyecciones_variable)

    ahorro_intereses_variable, intereses_sin_inyecciones_variable, intereses_con_inyecciones_variable = calcular_ahorro_intereses_variable_inyeccion(
        capital_inicial_variable_inyeccion, spread_variable_inyeccion, plazo_inicial_variable_inyeccion,
        cuota_inicial_variable_inyeccion, euribor_anual_values_ejemplo_inyeccion, inyecciones_variable)


    print("\n--- Simulación de Hipoteca Variable CON Inyecciones ---")
    print(f"Capital Inicial: {capital_inicial_variable_inyeccion:,.2f} euros")
    print(f"Spread: {spread_variable_inyeccion} %")
    print(f"Euribor: {euribor_anual_values_ejemplo_inyeccion} %")
    print(f"Plazo Inicial: {plazo_inicial_variable_inyeccion // 12} años")
    print(f"Cuota Inicial: {cuota_inicial_variable_inyeccion:.2f} euros")

    print("Inyecciones:") # Encabezado para inyecciones
    for inyeccion in inyecciones_variable: # Iterar sobre la lista de inyecciones
        print(f"  Mes: {inyeccion['mes_inyeccion']}, Capital: {inyeccion['capital_inyectado']:,.2f} €, Tipo: {inyeccion['tipo_inyeccion']}") # Imprimir cada inyección formateada


    print("Intereses")
    print(f"Ahorro total de intereses (hipoteca variable con inyecciones vs. original variable): {ahorro_intereses_variable:.2f} euros")
    print(f"Intereses totales pagados (hipoteca variable CON inyecciones): {intereses_con_inyecciones_variable:.2f} euros")
    print(f"Intereses totales pagados (hipoteca variable original SIN inyecciones): {intereses_sin_inyecciones_variable:.2f} euros")
    print("\nDataFrame de la simulación de hipoteca variable CON inyecciones:")
    #print(df_hipoteca_variable_con_inyecciones)


    # Simulación de hipoteca variable SIN inyecciones para comparar (para la gráfica)
    df_hipoteca_variable_sin_inyecciones = simulacion_hipoteca_variable_inyeccion(
        capital_inicial_variable_inyeccion, spread_variable_inyeccion, plazo_inicial_variable_inyeccion,
        cuota_inicial_variable_inyeccion, euribor_anual_values_ejemplo_inyeccion, inyecciones=[]) # Sin inyecciones


    plot_comparacion_variable(df_hipoteca_variable_sin_inyecciones, df_hipoteca_variable_con_inyecciones)


--- Simulación de Hipoteca Variable CON Inyecciones ---
Capital Inicial: 200,000.00 euros
Spread: 1.5 %
Euribor: [0.5, 0.7, 1.0, 1.2, 1.5, 1.3, 1.0, 0.8, 1.0, 1.2, 1.5, 1.8, 2.0, 2.2, 2.5, 2.3, 2.0, 1.8, 2.0, 2.2, 2.5, 2.8, 3.0, 3.2, 3.5, 3.3, 3.0, 2.8, 3.0, 3.2] %
Plazo Inicial: 30 años
Cuota Inicial: 739.24 euros
Inyecciones:
  Mes: 60, Capital: 10,000.00 €, Tipo: cuota
Intereses
Ahorro total de intereses (hipoteca variable con inyecciones vs. original variable): 4833.25 euros
Intereses totales pagados (hipoteca variable CON inyecciones): 102824.45 euros
Intereses totales pagados (hipoteca variable original SIN inyecciones): 107657.70 euros

DataFrame de la simulación de hipoteca variable CON inyecciones:


# Costes fijos:

* tasación
* notaria
* registro
* ITP
* gestoría

Links de interés:

- https://www.idealista.com/hipotecas/simulador-hipotecas/

- https://www.openbank.es/simulador


Para beneficiarse de las bonificaciones en el TAE (TIN) hay que cumplir lo siguiente:

* Contratar un seguro de vida (250€ / año)
* Contratar un seguro del hogar (202€ / año)

In [ ]:


list_costes_seguros_ajustados_inflacion = []
for i in range(1, 30):
  list_costes_seguros_ajustados_inflacion.append(452.07 * (1 + 0.001)**i)

sum(list_costes_seguros_ajustados_inflacion)

13308.52830787036

In [ ]:
# --- Estimaciones de Costes Fijos Iniciales de Hipoteca (VALORES ORIENTATIVOS) ---
# **IMPORTANTE:** Estos son solo ejemplos y deben ser revisados y ajustados con datos reales.

# Tasación (porcentaje del capital inicial - aunque en la realidad depende del valor del inmueble, lo simplificamos aquí)
COSTO_TASACION_MIN_EUROS = 200  # Estimación mínima en euros
COSTO_TASACION_MAX_EUROS = 500  # Estimación máxima en euros

# Gastos de Notaría (compraventa - porcentaje del precio de la vivienda, aquí lo simplificamos al capital inicial hipoteca)
GASTOS_NOTARIA_COMPRAVENTA_MIN_PORCENTAJE = 0.1  # Porcentaje mínimo
GASTOS_NOTARIA_COMPRAVENTA_MAX_PORCENTAJE = 0.5  # Porcentaje máximo

# Gastos de Registro de la Propiedad (compraventa - porcentaje del precio de la vivienda, simplificado al capital inicial)
GASTOS_REGISTRO_COMPRAVENTA_MIN_PORCENTAJE = 0.1 # Porcentaje mínimo
GASTOS_REGISTRO_COMPRAVENTA_MAX_PORCENTAJE = 0.2 # Porcentaje máximo

# Impuestos (ITP o IVA - porcentaje del precio de la vivienda, simplificado al capital inicial)
IMPUESTO_TRANSMISIONES_PATRIMONIALES_PORCENTAJE_MIN = 6.0 # Porcentaje mínimo ITP (puede variar por CCAA)
IMPUESTO_TRANSMISIONES_PATRIMONIALES_PORCENTAJE_MAX = 10.0 # Porcentaje máximo ITP (puede variar por CCAA)
IVA_VIVIENDA_NUEVA_PORCENTAJE = 10.0 # Porcentaje IVA vivienda nueva (general)
IVA_VIVIENDA_NUEVA_REDUCIDO_PORCENTAJE = 4.0 # Porcentaje IVA vivienda nueva (VPO)

# Gastos de Gestoría (importe fijo, si se contrata - opcional)
COSTO_GESTORIA_MIN_EUROS = 300 # Estimación mínima en euros (si se contrata)
COSTO_GESTORIA_MAX_EUROS = 600 # Estimación máxima en euros (si se contrata)


# --- Variables para la simulación de hipoteca (ya existentes) ---
# (Aquí seguiría el resto del código que ya teníamos: funciones, ejemplo de uso, etc.)

In [ ]:
def calcular_costes_iniciales_estimados(capital_inicial, es_vivienda_nueva=False, contratar_gestoria=False, impuesto_ccaa_itp_porcentaje=8.0):
    """
    Estima los costes iniciales de una hipoteca, basados en porcentajes y rangos definidos.

    Args:
        capital_inicial (float): Capital inicial de la hipoteca.
        es_vivienda_nueva (bool, optional): True si es vivienda nueva (aplica IVA), False si es de segunda mano (aplica ITP). Por defecto False.
        contratar_gestoria (bool, optional): True si se contrata gestoría, False si no. Por defecto False.
        impuesto_ccaa_itp_porcentaje (float, optional): Porcentaje de ITP de la Comunidad Autónoma (si es vivienda de segunda mano). Por defecto 8.0%.

    Returns:
        dict: Diccionario con el detalle de los costes iniciales estimados y el coste total.
    """

    coste_tasacion = (COSTO_TASACION_MIN_EUROS + COSTO_TASACION_MAX_EUROS) / 2 # Usamos el punto medio del rango como estimación
    coste_notaria_compraventa = capital_inicial * (GASTOS_NOTARIA_COMPRAVENTA_MIN_PORCENTAJE + GASTOS_NOTARIA_COMPRAVENTA_MAX_PORCENTAJE) / 200 # Punto medio del rango en porcentaje
    coste_registro_compraventa = capital_inicial * (GASTOS_REGISTRO_COMPRAVENTA_MIN_PORCENTAJE + GASTOS_REGISTRO_COMPRAVENTA_MAX_PORCENTAJE) / 200 # Punto medio del rango en porcentaje
    coste_gestoria = 0
    if contratar_gestoria:
        coste_gestoria = (COSTO_GESTORIA_MIN_EUROS + COSTO_GESTORIA_MAX_EUROS) / 2 # Punto medio del rango si se contrata

    if es_vivienda_nueva:
        coste_impuestos_compraventa = capital_inicial * IVA_VIVIENDA_NUEVA_PORCENTAJE / 100 # IVA general
        # Podríamos añadir lógica para IVA reducido VPO si es necesario
    else:
        coste_impuestos_compraventa = capital_inicial * impuesto_ccaa_itp_porcentaje / 100 # ITP

    costes_iniciales_detalle = {
        "Tasacion": coste_tasacion,
        "Notaria_Compraventa": coste_notaria_compraventa,
        "Registro_Compraventa": coste_registro_compraventa,
        "Impuestos_Compraventa": coste_impuestos_compraventa,
        "Gestoria": coste_gestoria
    }

    coste_total_inicial = sum(costes_iniciales_detalle.values())

    return costes_iniciales_detalle, coste_total_inicial

In [ ]:
    # --- Estimación de Costes Iniciales ---
    detalle_costes_iniciales, coste_total_inicial_estimado = calcular_costes_iniciales_estimados(
        capital_inicial=capital_inicial,
        es_vivienda_nueva=False, # Suponemos vivienda de segunda mano
        contratar_gestoria=True, # Suponemos que se contrata gestoría
        impuesto_ccaa_itp_porcentaje=8.0 # Suponemos ITP al 8% (valor de ejemplo)
    )

    print("\n--- Estimación de Costes Iniciales de Hipoteca ---")
    print(f"Capital Inicial Hipoteca: {capital_inicial} euros")
    print(f"Costes Iniciales Estimados (Detalle):")
    for nombre_coste, valor_coste in detalle_costes_iniciales.items():
        print(f"  - {nombre_coste}: {valor_coste:.2f} euros")
    print(f"Coste Inicial Total Estimado: {coste_total_inicial_estimado:.2f} euros")
    print(f"Coste Inicial Total Estimado como porcentaje del capital inicial: {coste_total_inicial_estimado / capital_inicial * 100:.2f}%")



--- Estimación de Costes Iniciales de Hipoteca ---
Capital Inicial Hipoteca: 149765 euros
Costes Iniciales Estimados (Detalle):
  - Tasacion: 350.00 euros
  - Notaria_Compraventa: 449.30 euros
  - Registro_Compraventa: 224.65 euros
  - Impuestos_Compraventa: 11981.20 euros
  - Gestoria: 450.00 euros
Coste Inicial Total Estimado: 13455.14 euros
Coste Inicial Total Estimado como porcentaje del capital inicial: 8.98%


# Simulaciones

In [ ]:
def simulacion_hipoteca_variable_distribucion_euribor(capital_inicial, spread, plazo_inicial,
                                                        cuota_inicial, euribor_anual_distribucion):
    """
    Simulación de hipoteca variable con distribución probabilística para el Euribor.

    Args:
        capital_inicial (float): Capital inicial del préstamo.
        spread (float): Diferencial (spread) sobre el Euribor en porcentaje.
        plazo_inicial (int): Plazo inicial del préstamo en meses.
        cuota_inicial (float): Cuota mensual inicial (calculada con el primer Euribor *de la distribución*).
        euribor_anual_distribucion (function): Función que genera un valor aleatorio del Euribor anual.
                                                Debe ser una función sin argumentos que retorne un valor numérico
                                                representando el Euribor anual en porcentaje.
                                                Ejemplo:  lambda: np.random.normal(loc=1.0, scale=0.5) # Distribución Normal

    Returns:
        pd.DataFrame: DataFrame con el detalle de la simulación mes a mes.
    """

    # Validaciones iniciales generales
    if capital_inicial <= 0:
        raise ValueError("El capital inicial debe ser positivo")
    if plazo_inicial <= 0:
        raise ValueError("El plazo inicial debe ser positivo")
    if cuota_inicial <= 0:
        raise ValueError("La cuota inicial debe ser positiva")
    if not callable(euribor_anual_distribucion):
        raise TypeError("euribor_anual_distribucion debe ser una función (lambda o función definida).")


    registros = []
    capital_pendiente = capital_inicial
    cuota_mensual_fija = cuota_inicial  # Inicialmente fijada, puede recalcularse
    mes_actual = 0
    plazo_restante = plazo_inicial # Plazo restante que se irá actualizando
    year_actual = 0 # Para controlar el cambio de año y aplicar el Euribor correspondiente
    euribor_anual_values_simulacion = [] # Lista para guardar los valores de Euribor simulados para esta ejecución


    for mes in range(1, plazo_inicial + 1):
        mes_actual += 1
        year_in_simulation = math.ceil(mes_actual / 12) # Calcula el año en la simulación

        # Actualizar tipo de interés anualmente según la distribución del Euribor
        if year_in_simulation > year_actual and year_in_simulation <= plazo_inicial // 12 + 1: # Asegurar que no se generen más valores de Euribor de los necesarios
            year_actual = year_in_simulation
            euribor_anual_actual = max(euribor_anual_distribucion(), 0) # Obtener valor de Euribor de la distribución, asegurando que no sea negativo
            euribor_anual_values_simulacion.append(euribor_anual_actual) # Guardar valor simulado
            tasa_anual_actual = euribor_anual_actual + spread # Euribor + diferencial
            cuota_mensual_fija = cuota_mensual(capital_pendiente, tasa_anual_actual, plazo_restante) # Recalcular cuota con nuevo tipo y capital pendiente
            #print(f"Año {year_actual}, Euribor Simulado: {euribor_anual_actual:.2f}%, Tasa Anual Actual: {tasa_anual_actual:.2f}%, Nueva Cuota: {cuota_mensual_fija:.2f}")


        elif year_in_simulation > plazo_inicial // 12 + 1: # Si se supera el plazo inicial en años (por redondeo hacia arriba en el bucle)
            tasa_anual_actual = tasa_anual_actual # Mantiene la última tasa

        elif year_actual == 0 and year_in_simulation == 1: # Primer año
             euribor_anual_actual = max(euribor_anual_distribucion(), 0) # Primer valor de Euribor de la distribución, no negativo
             euribor_anual_values_simulacion.append(euribor_anual_actual) # Guardar valor simulado
             tasa_anual_actual = euribor_anual_actual + spread


        if capital_pendiente <= 0:
            registros.append({
                'Mes': mes_actual,
                'Año_Simulacion': year_in_simulation,
                'Euribor_Anual': euribor_anual_actual if 'euribor_anual_actual' in locals() else 0,
                'Tasa_Anual': tasa_anual_actual if 'tasa_anual_actual' in locals() else 0,
                'Capital_pendiente': 0,
                'Cuota_mensual': 0,
                'Intereses_mensuales': 0,
                'Amortizacion_mensual': 0
            })
            break  # Finalizar si ya se pagó todo

        interes = intereses_mensuales(capital_pendiente, tasa_anual_actual) # Usar la tasa anual actual
        amortizacion = min(cuota_mensual_fija - interes, capital_pendiente)  # No sobrepagar

        registros.append({
            'Mes': mes_actual,
            'Año_Simulacion': year_in_simulation,
            'Euribor_Anual': euribor_anual_actual if 'euribor_anual_actual' in locals() else 0,
            'Tasa_Anual': tasa_anual_actual if 'tasa_anual_actual' in locals() else 0,
            'Capital_pendiente': capital_pendiente,
            'Cuota_mensual': cuota_mensual_fija,
            'Intereses_mensuales': interes,
            'Amortizacion_mensual': amortizacion
        })

        capital_pendiente -= amortizacion
        plazo_restante -=1 # Reducir el plazo restante cada mes


    return pd.DataFrame(registros), euribor_anual_values_simulacion # Devuelve también los valores de Euribor simulados


def ejecutar_simulaciones_multiples(num_simulaciones, capital_inicial, spread, plazo_inicial,
                                       cuota_inicial, euribor_anual_distribucion):
    """
    Ejecuta múltiples simulaciones de hipoteca variable y retorna los resultados agregados.

    Args:
        num_simulaciones (int): Número de simulaciones a ejecutar.
        capital_inicial (float): Capital inicial del préstamo.
        spread (float): Diferencial sobre el Euribor.
        plazo_inicial (int): Plazo inicial en meses.
        cuota_inicial (float): Cuota mensual inicial (calculada con un valor inicial de Euribor).
        euribor_anual_distribucion (function): Función de distribución del Euribor anual.

    Returns:
        dict: Diccionario con DataFrames y valores de Euribor de todas las simulaciones.
              Ej: {'simulacion_1': {'df': DataFrame_1, 'euribor_values': [val1, val2, ...]}, ...}
    """
    resultados_simulaciones = {}
    for i in range(1, num_simulaciones + 1):
        df_simulacion, euribor_values = simulacion_hipoteca_variable_distribucion_euribor(
            capital_inicial, spread, plazo_inicial, cuota_inicial, euribor_anual_distribucion)
        resultados_simulaciones[f'simulacion_{i}'] = {'df': df_simulacion, 'euribor_values': euribor_values}
    return resultados_simulaciones



def plot_distribucion_cuotas(resultados_simulaciones, mes_a_visualizar=12):
    """
    Genera un gráfico de distribución de cuotas mensuales para un mes específico a partir de múltiples simulaciones.

    Args:
        resultados_simulaciones (dict): Diccionario con resultados de simulaciones (output de ejecutar_simulaciones_multiples).
        mes_a_visualizar (int): Mes para el cual se visualizará la distribución de cuotas.
    """

    cuotas_mes = []
    for sim_data in resultados_simulaciones.values():
        df_sim = sim_data['df']
        if mes_a_visualizar in df_sim['Mes'].values: # Asegurar que el mes exista en la simulación
            cuota_mes_sim = df_sim[df_sim['Mes'] == mes_a_visualizar]['Cuota_mensual'].iloc[0]
            cuotas_mes.append(cuota_mes_sim)

    if not cuotas_mes:
        print(f"No hay datos de cuota para el mes {mes_a_visualizar} en las simulaciones.")
        return

    fig = go.Figure(data=[go.Histogram(x=cuotas_mes, nbinsx=50)]) # Ajustar nbinsx para resolución del histograma
    fig.update_layout(
        title=f'Distribución de Cuotas Mensuales para el Mes {mes_a_visualizar} (Simulaciones Hipoteca Variable)',
        xaxis_title=f'Cuota Mensual (euros) - Mes {mes_a_visualizar}',
        yaxis_title='Frecuencia',
        bargap=0.2, # Espacio entre barras del histograma
        font_family="Courier New",
        font_color="black",
        title_font_family="Times New Roman",
        title_font_color="black",
        legend_title_font_color="black",
    )
    fig.show()


def plot_heatmap_cuotas_probabilidad(resultados_simulaciones, meses_a_visualizar=None, num_bins_cuota=30, colorscale_name='Viridis'):
    """
    Genera un heatmap de probabilidad de cuotas mensuales a lo largo del tiempo,
    con opción para elegir la escala de color (colorscale).

    Args:
        resultados_simulaciones (dict): Diccionario con resultados de simulaciones.
        meses_a_visualizar (list, optional): Lista de meses a incluir en el heatmap. Si None, se usan todos los meses simulados.
        num_bins_cuota (int): Número de bins para discretizar el rango de cuotas en el heatmap.
        colorscale_name (str): Nombre de la colorscale de Plotly a utilizar (ej: 'Viridis', 'Hot', 'Plasma', 'Greys').
    """

    all_months = set()
    all_cuotas_mes_data = {}

    for sim_data in resultados_simulaciones.values():
        df_sim = sim_data['df']
        months_in_sim = df_sim['Mes'].unique()
        all_months.update(months_in_sim)
        for month in months_in_sim:
            if meses_a_visualizar is None or month in meses_a_visualizar:
                cuota_mes_sim = df_sim[df_sim['Mes'] == month]['Cuota_mensual'].iloc[0]
                if month not in all_cuotas_mes_data:
                    all_cuotas_mes_data[month] = []
                all_cuotas_mes_data[month].append(cuota_mes_sim)

    meses_ordenados = sorted(list(all_months))
    if meses_a_visualizar:
        meses_ordenados = sorted(list(set(meses_ordenados) & set(meses_a_visualizar)))

    if not meses_ordenados:
        print("No hay datos para los meses especificados o en las simulaciones.")
        return

    # Preparar datos para el heatmap
    cuotas_rangos = np.linspace(min(min(cuotas) for cuotas in all_cuotas_mes_data.values()),
                                 max(max(cuotas) for cuotas in all_cuotas_mes_data.values()),
                                 num_bins_cuota) # Rangos de cuotas discretizados
    frecuencia_cuotas_mes = [] # Matriz para el heatmap: filas=meses, columnas=rangos de cuota

    for mes in meses_ordenados:
        hist, _ = np.histogram(all_cuotas_mes_data[mes], bins=cuotas_rangos)
        frecuencia_cuotas_mes.append(hist.tolist()) # Frecuencia de cuotas por rango para cada mes


    fig = go.Figure(data=go.Heatmap(
        z=frecuencia_cuotas_mes,
        x=cuotas_rangos.tolist(), # Rangos de cuotas como etiquetas del eje x
        y=meses_ordenados, # Meses como etiquetas del eje y
        colorscale=colorscale_name, # Usar la colorscale especificada como argumento
        colorbar=dict(title='Frecuencia') # Título de la barra de color
    ))

    fig.update_layout(
        title=f'Probabilidad de Rangos de Cuotas Mensuales a lo Largo del Tiempo (Simulaciones Hipoteca Variable - Colorscale: {colorscale_name})',
        xaxis_title='Rango de Cuota Mensual (euros)',
        yaxis_title='Mes',
        font_family="Courier New",
        font_color="black",
        title_font_family="Times New Roman",
        title_font_color="black",
        legend_title_font_color="black",
    )

    fig.show()

def plot_boxplot_cuotas(resultados_simulaciones, meses_a_visualizar=None):
    """
    Genera gráficos de caja (box plots) de las cuotas mensuales para cada mes simulado.

    Args:
        resultados_simulaciones (dict): Diccionario con resultados de simulaciones.
        meses_a_visualizar (list, optional): Lista de meses a incluir en los box plots. Si None, se usan todos los meses simulados.
    """

    cuotas_por_mes = {}
    all_months = set()

    for sim_data in resultados_simulaciones.values():
        df_sim = sim_data['df']
        months_in_sim = df_sim['Mes'].unique()
        all_months.update(months_in_sim)
        for month in months_in_sim:
            if meses_a_visualizar is None or month in meses_a_visualizar:
                cuota_mes_sim = df_sim[df_sim['Mes'] == month]['Cuota_mensual'].iloc[0]
                if month not in cuotas_por_mes:
                    cuotas_por_mes[month] = []
                cuotas_por_mes[month].append(cuota_mes_sim)

    meses_ordenados = sorted(list(all_months))
    if meses_a_visualizar:
        meses_ordenados = sorted(list(set(meses_ordenados) & set(meses_a_visualizar)))

    if not meses_ordenados:
        print("No hay datos para los meses especificados o en las simulaciones.")
        return

    data_boxplots = []
    for mes in meses_ordenados:
        data_boxplots.append(go.Box(y=cuotas_por_mes[mes], name=str(mes))) # Boxplot para cada mes

    fig = go.Figure(data=data_boxplots)

    fig.update_layout(
        title='Distribución de Cuotas Mensuales (Gráficos de Caja) - Simulación Hipoteca Variable',
        yaxis_title='Cuota Mensual (euros)',
        xaxis_title='Mes',
        font_family="Courier New",
        font_color="black",
        title_font_family="Times New Roman",
        title_font_color="black",
        legend_title_font_color="black",
    )

    fig.show()

def plot_violinplot_cuotas(resultados_simulaciones, meses_a_visualizar=None):
    """
    Genera gráficos de violín (violin plots) de las cuotas mensuales para cada mes simulado.

    Args:
        resultados_simulaciones (dict): Diccionario con resultados de simulaciones.
        meses_a_visualizar (list, optional): Lista de meses a incluir en los violin plots. Si None, se usan todos los meses simulados.
    """

    cuotas_por_mes = {}
    all_months = set()

    for sim_data in resultados_simulaciones.values():
        df_sim = sim_data['df']
        months_in_sim = df_sim['Mes'].unique()
        all_months.update(months_in_sim)
        for month in months_in_sim:
            if meses_a_visualizar is None or month in meses_a_visualizar:
                cuota_mes_sim = df_sim[df_sim['Mes'] == month]['Cuota_mensual'].iloc[0]
                if month not in cuotas_por_mes:
                    cuotas_por_mes[month] = []
                cuotas_por_mes[month].append(cuota_mes_sim)

    meses_ordenados = sorted(list(all_months))
    if meses_a_visualizar:
        meses_ordenados = sorted(list(set(meses_ordenados) & set(meses_a_visualizar)))

    if not meses_ordenados:
        print("No hay datos para los meses especificados o en las simulaciones.")
        return

    data_violins = []
    for mes in meses_ordenados:
        data_violins.append(go.Violin(y=cuotas_por_mes[mes], name=str(mes), box_visible=True, meanline_visible=True)) # Violin para cada mes

    fig = go.Figure(data=data_violins)

    fig.update_layout(
        title='Distribución de Cuotas Mensuales (Gráficos de Violín) - Simulación Hipoteca Variable',
        yaxis_title='Cuota Mensual (euros)',
        xaxis_title='Mes',
        font_family="Courier New",
        font_color="black",
        title_font_family="Times New Roman",
        title_font_color="black",
        legend_title_font_color="black",
    )

    fig.show()


if __name__ == '__main__':
    # --- Definición de parámetros de la hipoteca ---
    capital_inicial_simulacion = 200000
    spread_simulacion = 1.5 # Diferencial del 1.5%
    plazo_inicial_simulacion = 30 * 12 # 30 años en meses

    # --- Definición de la distribución del Euribor ---
    # Ejemplo: Distribución Normal con media 1.0% y desviación estándar 0.7%. AJUSTAR ESTOS VALORES CON DATOS REALES.
    euribor_distribucion_normal = lambda: np.random.normal(loc=1.0, scale=0.7)

    # --- Calcular la cuota inicial con un valor de Euribor de la distribución (puedes usar la media de la distribución para un valor "típico") ---
    euribor_inicial_simulacion = np.mean([euribor_distribucion_normal() for _ in range(10000)]) # Usar la media de la distribución como Euribor inicial aproximado
    tasa_anual_inicial_simulacion = euribor_inicial_simulacion + spread_simulacion
    cuota_inicial_simulacion = cuota_mensual(capital_inicial_simulacion, tasa_anual_inicial_simulacion, plazo_inicial_simulacion)


    # --- Ejecutar múltiples simulaciones ---
    num_simulaciones = 4000  # Aumenta este número para obtener una distribución más suave y representativa (ej: 1000, 5000...)
    resultados_simulaciones = ejecutar_simulaciones_multiples(num_simulaciones, capital_inicial_simulacion, spread_simulacion,
                                                                plazo_inicial_simulacion, cuota_inicial_simulacion, euribor_distribucion_normal)

    print(f"\n--- Simulación Múltiple de Hipoteca Variable ({num_simulaciones} simulaciones) ---")
    #print(f"Spread utilizado: {spread_simulacion}%, Distribución Euribor: Normal(loc={np.mean([euribor_distribucion_normal()() for _ in range(10000)]):.2f}%, scale={0.7:.2f}%)") # Muestra media aproximada de la distribucion
    print(f"Capital Inicial: {capital_inicial_simulacion} euros, Plazo Inicial: {plazo_inicial_simulacion // 12} años")
    print(f"Cuota Inicial (aprox. basada en media de Euribor): {cuota_inicial_simulacion:.2f} euros")

    # --- Graficar distribución de cuotas para un mes específico (ejemplo: mes 60) ---
    plot_distribucion_cuotas(resultados_simulaciones, mes_a_visualizar=60)
    # [Image of Histogram of monthly payments for month 60]

        # --- Graficar heatmap con diferentes colorscale para comparar ---
    colorscales_a_probar = [#'Viridis',
                            'Hot',
                            #'Plasma',
                            #'Magma',
                            #'Inferno',
                            #'Greys'
                            ] # Lista de colorscales a probar

    for colorscale_nombre in colorscales_a_probar:
        plot_heatmap_cuotas_probabilidad(resultados_simulaciones,
                                            meses_a_visualizar=range(1, 361, 12), # Heatmap para los primeros 60 meses
                                            num_bins_cuota=50,
                                            colorscale_name=colorscale_nombre)

    # --- Graficar heatmap de probabilidad de cuotas a lo largo del tiempo ---
    #plot_heatmap_cuotas_probabilidad(resultados_simulaciones, meses_a_visualizar=range(0, 361, 12), num_bins_cuota=50)
    # [Image of Heatmap of probability distribution of monthly payments over time]

    # --- Graficar Box Plots de cuotas mensuales ---
    plot_boxplot_cuotas(resultados_simulaciones, meses_a_visualizar=range(1, 361, 36)) # Box plots para los primeros 60 meses, cada 6 meses
    # [Image of Box plots of monthly payments]

    # --- Graficar Violin Plots de cuotas mensuales ---
    plot_violinplot_cuotas(resultados_simulaciones, meses_a_visualizar=range(1, 361, 36)) # Violin plots para los primeros 60 meses, cada 6 meses
    # [Image of Violin plots of monthly payments]

    # plot_heatmap_cuotas_probabilidad(resultados_simulaciones, meses_a_visualizar=range(1, 61), num_bins_cuota=50) # Heatmap para los primeros 60 meses
    # [Image of Heatmap of probability distribution of monthly payments for the first 60 months]


--- Simulación Múltiple de Hipoteca Variable (4000 simulaciones) ---
Capital Inicial: 200000 euros, Plazo Inicial: 30 años
Cuota Inicial (aprox. basada en media de Euribor): 790.17 euros


# Simulaciones hipoteca mixta.

- dejar fijo el interés durante 3, 5, 7 años. Después, simular euribor + diferencial partiendo de distribución gaussiana.

- simular varios escenarios de amortizaciones en plazo. (Optimista: 60K cada año), 50k, 40k, 30k, 20k, 10k.

- tirar varias simulaciones distintas para ver el impacto del euribor.

In [2]:
import math
import pandas as pd
import plotly.graph_objects as go
import numpy as np

def cuota_mensual(capital, tasa, plazo):
    """Calcula la cuota mensual con validación de parámetros"""
    if plazo <= 0:
        raise ValueError("El plazo debe ser mayor a 0 meses")
    if capital <= 0:
        return 0  # Si no hay capital, no hay cuota

    tasa_mensual = tasa / 1200
    return (capital * tasa_mensual) / (1 - (1 + tasa_mensual)**-plazo)

def intereses_mensuales(capital_pendiente, tasa_anual):
    """Calcula los intereses mensuales"""
    tasa_mensual = tasa_anual / 1200
    return capital_pendiente * tasa_mensual

def simulacion_hipoteca_mixta(capital_inicial, periodo_fijo_años, tasa_fija_anual, spread, plazo_inicial, euribor_anual_distribucion):
    """
    Simulación de hipoteca mixta con periodo fijo y luego variable (Euribor + diferencial).

    Args:
        capital_inicial (float): Capital inicial del préstamo.
        periodo_fijo_años (int): Duración del periodo fijo en años.
        tasa_fija_anual (float): Tasa de interés anual fija durante el periodo fijo en porcentaje.
        spread (float): Diferencial (spread) sobre el Euribor en porcentaje.
        plazo_inicial (int): Plazo inicial del préstamo en meses.
        euribor_anual_distribucion (function): Función que genera un valor aleatorio del Euribor anual.

    Returns:
        pd.DataFrame: DataFrame con el detalle de la simulación mes a mes.
    """

    # Validaciones iniciales generales
    if capital_inicial <= 0:
        raise ValueError("El capital inicial debe ser positivo")
    if plazo_inicial <= 0:
        raise ValueError("El plazo inicial debe ser positivo")
    if periodo_fijo_años < 0:
        raise ValueError("El periodo fijo en años no puede ser negativo")
    if not callable(euribor_anual_distribucion):
        raise TypeError("euribor_anual_distribucion debe ser una función.")

    registros = []
    capital_pendiente = capital_inicial
    mes_actual = 0
    plazo_restante = plazo_inicial
    year_actual = 0
    euribor_anual_values_simulacion = []

    for mes in range(1, plazo_inicial + 1):
        mes_actual += 1
        year_in_simulation = math.ceil(mes_actual / 12)

        # --- Calcular la tasa de interés actual ---
        if year_in_simulation <= periodo_fijo_años:
            tasa_anual_actual = tasa_fija_anual
        else:
            if year_in_simulation > year_actual and year_in_simulation <= plazo_inicial // 12 + 1:
                year_actual = year_in_simulation
                euribor_anual_actual = max(euribor_anual_distribucion(), 0)
                euribor_anual_values_simulacion.append(euribor_anual_actual)
                tasa_anual_actual = euribor_anual_actual + spread
            elif year_in_simulation > plazo_inicial // 12 + 1:
                pass # Mantiene la última tasa
            elif year_actual == 0 and year_in_simulation == 1 and periodo_fijo_años == 0: # Si el periodo fijo es 0, empezar con variable
                euribor_anual_actual = max(euribor_anual_distribucion(), 0)
                euribor_anual_values_simulacion.append(euribor_anual_actual)
                tasa_anual_actual = euribor_anual_actual + spread
            elif year_actual == 0 and year_in_simulation == 1 and periodo_fijo_años > 0:
                tasa_anual_actual = tasa_fija_anual
            elif year_in_simulation > periodo_fijo_años: # Asegurar que se actualiza el euribor tras el periodo fijo
                if 'euribor_anual_actual' not in locals():
                    euribor_anual_actual = max(euribor_anual_distribucion(), 0)
                    euribor_anual_values_simulacion.append(euribor_anual_actual)
                tasa_anual_actual = euribor_anual_actual + spread


        # --- Calcular la cuota mensual ---
        if mes == 1: # Calcular la cuota inicial al inicio
            cuota_mensual_fija = cuota_mensual(capital_inicial, tasa_anual_actual, plazo_inicial)
        elif year_in_simulation == periodo_fijo_años + 1 and periodo_fijo_años > 0: # Recalcular cuota al inicio del periodo variable
            cuota_mensual_fija = cuota_mensual(capital_pendiente, tasa_anual_actual, plazo_restante)
        elif year_in_simulation > periodo_fijo_años + 1: # Recalcular cuota anualmente en el periodo variable
            if (mes -1) % 12 == 0:
                cuota_mensual_fija = cuota_mensual(capital_pendiente, tasa_anual_actual, plazo_restante)


        if capital_pendiente <= 0:
            registros.append({
                'Mes': mes_actual,
                'Año_Simulacion': year_in_simulation,
                'Periodo': 'Fijo' if year_in_simulation <= periodo_fijo_años else 'Variable',
                'Euribor_Anual': euribor_anual_actual if 'euribor_anual_actual' in locals() else 0,
                'Tasa_Anual': tasa_anual_actual,
                'Capital_pendiente': 0,
                'Cuota_mensual': 0,
                'Intereses_mensuales': 0,
                'Amortizacion_mensual': 0
            })
            break

        interes = intereses_mensuales(capital_pendiente, tasa_anual_actual)
        amortizacion = min(cuota_mensual_fija - interes, capital_pendiente)

        registros.append({
            'Mes': mes_actual,
            'Año_Simulacion': year_in_simulation,
            'Periodo': 'Fijo' if year_in_simulation <= periodo_fijo_años else 'Variable',
            'Euribor_Anual': euribor_anual_actual if 'euribor_anual_actual' in locals() else 0,
            'Tasa_Anual': tasa_anual_actual,
            'Capital_pendiente': capital_pendiente,
            'Cuota_mensual': cuota_mensual_fija,
            'Intereses_mensuales': interes,
            'Amortizacion_mensual': amortizacion
        })

        capital_pendiente -= amortizacion
        plazo_restante -= 1

    return pd.DataFrame(registros), euribor_anual_values_simulacion

def simular_amortizacion_anticipada(df_simulacion, amortizacion_anual):
    """
    Aplica amortizaciones anticipadas anuales a una simulación de hipoteca.

    Args:
        df_simulacion (pd.DataFrame): DataFrame resultante de la simulación de hipoteca.
        amortizacion_anual (float): Cantidad anual a amortizar anticipadamente.

    Returns:
        pd.DataFrame: DataFrame con la simulación actualizada incluyendo las amortizaciones anticipadas.
    """
    df_amortizado = df_simulacion.copy()
    capital_pendiente = df_amortizado['Capital_pendiente'].iloc[0]
    mes_actual = 0
    amortizacion_acumulada = 0

    for index, row in df_amortizado.iterrows():
        mes_actual = int(row['Mes'])
        año_simulacion = int(row['Año_Simulacion'])

        # Aplicar amortización al final de cada año
        if mes_actual > 0 and mes_actual % 12 == 0:
            amortizar = min(amortizacion_anual, capital_pendiente)
            capital_pendiente -= amortizar
            df_amortizado.loc[index, 'Capital_pendiente'] = capital_pendiente
            df_amortizado.loc[index, 'Amortizacion_mensual'] += amortizar # Sumar a la amortización regular
            amortizacion_acumulada += amortizar

        # Recalcular intereses para el siguiente mes
        if index + 1 < len(df_amortizado):
            tasa_anual = df_amortizado.loc[index + 1, 'Tasa_Anual']
            interes_nuevo = intereses_mensuales(capital_pendiente, tasa_anual)
            df_amortizado.loc[index + 1, 'Intereses_mensuales'] = interes_nuevo
            # Recalcular la amortización del siguiente mes (la cuota se mantiene fija hasta la revisión)
            cuota = df_amortizado.loc[index + 1, 'Cuota_mensual']
            amortizacion_nueva = min(cuota - interes_nuevo, capital_pendiente)
            df_amortizado.loc[index + 1, 'Amortizacion_mensual'] = amortizacion_nueva
            capital_pendiente -= amortizacion_nueva


    return df_amortizado

def calcular_intereses_totales(df_simulacion):
    """Calcula los intereses totales pagados durante la vida del préstamo."""
    intereses_totales = df_simulacion['Intereses_mensuales'].sum()
    return intereses_totales

def plot_evolucion_capital(df_simulacion, titulo='Evolución del Capital Pendiente'):
    """Grafica la evolución del capital pendiente a lo largo del tiempo."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_simulacion['Mes'], y=df_simulacion['Capital_pendiente'], mode='lines'))
    fig.update_layout(title=titulo, xaxis_title='Mes', yaxis_title='Capital Pendiente (euros)')
    fig.show()

if __name__ == '__main__':
    # --- Definición de parámetros de la hipoteca ---
    capital_inicial = 150000
    spread = 1.5 # Diferencial del 1.5%
    plazo_inicial = 30 * 12 # 30 años en meses
    tasas_fijas = [2.0, 2.5, 3.0] # Tasas fijas a probar
    periodos_fijos = [3, 5, 7] # Periodos fijos a probar
    amortizaciones_anuales = [60000, 50000, 40000, 30000, 20000, 10000, 0] # Amortizaciones anuales a simular
    num_simulaciones_euribor = 5 # Número de simulaciones para cada escenario de Euribor

    # --- Definición de la distribución del Euribor ---
    # Ejemplo: Distribución Normal con media 3.0% y desviación estándar 0.7%. AJUSTAR CON DATOS REALES.
    euribor_distribucion_normal = lambda: np.random.normal(loc=3.0, scale=0.7)

    resultados_globales = {}

    for periodo_fijo in periodos_fijos:
        for tasa_fija in tasas_fijas:
            escenario = f"Fijo {periodo_fijo} años ({tasa_fija:.2f}%)"
            resultados_globales[escenario] = {}
            print(f"\n--- Escenario: {escenario} ---")

            # Simular la parte variable para obtener una cuota inicial aproximada (si el periodo fijo es menor que el plazo total)
            if periodo_fijo < plazo_inicial // 12:
                # Simular un valor de Euribor para calcular la cuota inicial del periodo fijo
                euribor_inicial = np.mean([euribor_distribucion_normal() for _ in range(1000)])
                tasa_inicial = tasa_fija # Usar la tasa fija para la cuota inicial
                cuota_inicial = cuota_mensual(capital_inicial, tasa_inicial, plazo_inicial)
            else:
                cuota_inicial = cuota_mensual(capital_inicial, tasa_fija, plazo_inicial) # Si todo es fijo

            for amortizacion in amortizaciones_anuales:
                resultados_globales[escenario][f"Amortización {amortizacion if amortizacion > 0 else 'Sin'}"] = []
                print(f"\n  --- Amortización Anual: {amortizacion if amortizacion > 0 else 'Sin'} ---")

                intereses_totales_simulaciones = []

                for i in range(num_simulaciones_euribor):
                    df_sim, euribor_values = simulacion_hipoteca_mixta(
                        capital_inicial, periodo_fijo, tasa_fija, spread, plazo_inicial, euribor_distribucion_normal
                    )

                    df_sim_amortizado = df_sim
                    if amortizacion > 0:
                        df_sim_amortizado = simular_amortizacion_anticipada(df_sim, amortizacion)

                    intereses_totales = calcular_intereses_totales(df_sim_amortizado)
                    intereses_totales_simulaciones.append(intereses_totales)
                    resultados_globales[escenario][f"Amortización {amortizacion if amortizacion > 0 else 'Sin'}"].append(df_sim_amortizado)

                print(f"    Intereses Totales (promedio de {num_simulaciones_euribor} simulaciones): {np.mean(intereses_totales_simulaciones):.2f} euros")

                # --- Visualización para un escenario específico (puedes modificar esto para ver otros) ---
                if periodo_fijo == 5 and tasa_fija == 2.5 and amortizacion == 30000 and num_simulaciones_euribor > 0:
                    plot_evolucion_capital(resultados_globales[escenario][f"Amortización {amortizacion}"][0],
                                            titulo=f"Evolución del Capital Pendiente - {escenario} - Amortización {amortizacion}")


--- Escenario: Fijo 3 años (2.00%) ---

  --- Amortización Anual: 60000 ---
    Intereses Totales (promedio de 5 simulaciones): 5034.87 euros

  --- Amortización Anual: 50000 ---
    Intereses Totales (promedio de 5 simulaciones): 5644.47 euros

  --- Amortización Anual: 40000 ---
    Intereses Totales (promedio de 5 simulaciones): 6618.45 euros

  --- Amortización Anual: 30000 ---
    Intereses Totales (promedio de 5 simulaciones): 8283.66 euros

  --- Amortización Anual: 20000 ---
    Intereses Totales (promedio de 5 simulaciones): 11182.96 euros

  --- Amortización Anual: 10000 ---
    Intereses Totales (promedio de 5 simulaciones): 19312.58 euros

  --- Amortización Anual: Sin ---
    Intereses Totales (promedio de 5 simulaciones): 63912.32 euros

--- Escenario: Fijo 3 años (2.50%) ---

  --- Amortización Anual: 60000 ---
    Intereses Totales (promedio de 5 simulaciones): 6304.31 euros

  --- Amortización Anual: 50000 ---
    Intereses Totales (promedio de 5 simulaciones): 7069.3


  --- Amortización Anual: 20000 ---
    Intereses Totales (promedio de 5 simulaciones): 13049.77 euros

  --- Amortización Anual: 10000 ---
    Intereses Totales (promedio de 5 simulaciones): 20655.03 euros

  --- Amortización Anual: Sin ---
    Intereses Totales (promedio de 5 simulaciones): 65298.82 euros

--- Escenario: Fijo 5 años (3.00%) ---

  --- Amortización Anual: 60000 ---
    Intereses Totales (promedio de 5 simulaciones): 7575.99 euros

  --- Amortización Anual: 50000 ---
    Intereses Totales (promedio de 5 simulaciones): 8497.72 euros

  --- Amortización Anual: 40000 ---
    Intereses Totales (promedio de 5 simulaciones): 9828.56 euros

  --- Amortización Anual: 30000 ---
    Intereses Totales (promedio de 5 simulaciones): 11938.19 euros

  --- Amortización Anual: 20000 ---
    Intereses Totales (promedio de 5 simulaciones): 15740.51 euros

  --- Amortización Anual: 10000 ---
    Intereses Totales (promedio de 5 simulaciones): 24381.47 euros

  --- Amortización Anual: Si

In [6]:
if __name__ == '__main__':

 # --- Guardar los resultados en un archivo Excel ---
    nombre_archivo_excel = "simulacion_hipoteca.xlsx"
    with pd.ExcelWriter(nombre_archivo_excel) as writer:
        for escenario, amortizaciones in resultados_globales.items():
            for amortizacion_desc, lista_dfs in amortizaciones.items():
                for i, df_simulacion in enumerate(lista_dfs):
                    nombre_hoja = f"{escenario.replace(' ', '_')}_{amortizacion_desc.replace(' ', '_')}_Simulacion_{i+1}"
                    df_simulacion.to_excel(writer, sheet_name=nombre_hoja, index=False)

    print(f"\nLos resultados de la simulación se han guardado en el archivo: {nombre_archivo_excel}")

/usr/local/lib/python3.11/dist-packages/openpyxl/workbook/child.py:99: UserWarning:

Title is more than 31 characters. Some applications may not be able to read the file




Los resultados de la simulación se han guardado en el archivo: simulacion_hipoteca.xlsx
